In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv(r'C:\Users\m_joekid\Desktop\Vscodes\datasets\Data_for_UCI_named.csv')

## Descriptive Stats

In [3]:
df.head(2)

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable


In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
tau1,10000.0,5.250000,2.742548,0.500793,2.874892,5.250004,7.624690,9.999469
tau2,10000.0,5.250001,2.742549,0.500141,2.875140,5.249981,7.624893,9.999837
tau3,10000.0,5.250004,2.742549,0.500788,2.875522,5.249979,7.624948,9.999450
tau4,10000.0,5.249997,2.742556,0.500473,2.874950,5.249734,7.624838,9.999443
p1,10000.0,3.750000,0.752160,1.582590,3.218300,3.751025,4.282420,5.864418
p2,10000.0,-1.250000,0.433035,-1.999891,-1.624901,-1.249966,-0.874977,-0.500108
p3,10000.0,-1.250000,0.433035,-1.999945,-1.625025,-1.249974,-0.875043,-0.500072
p4,10000.0,-1.250000,0.433035,-1.999926,-1.624960,-1.250007,-0.875065,-0.500025
g1,10000.0,0.525000,0.274256,0.050009,0.287521,0.525009,0.762435,0.999937
g2,10000.0,0.525000,0.274255,0.050053,0.287552,0.525003,0.762490,0.999944


##  | |

In [5]:
# only one dependent variable is kept since both have direct relationship 
df.drop('stab',inplace=True,axis=1)

In [6]:
# Extreme boosting and light gradient boosting libary  
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [7]:
# X contains the predictive feautures and y the dependent variable
X=df.drop('stabf',axis=1)
y=df['stabf']

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
# splitting the dataset 80:20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [10]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report,confusion_matrix
from sklearn import metrics

In [11]:
scaler=StandardScaler()

In [12]:
# Defining model hyperparameters
rf=RandomForestClassifier(random_state=1,n_estimators=100)
extra=ExtraTreesClassifier(random_state=1,n_estimators=100)
xgb=XGBClassifier(random_state=1)
lgb=LGBMClassifier(random_state=1)

In [13]:
# Creating model pipelines that scales and fits the data
rf_pipe=make_pipeline(scaler,rf)
extra_pipe=make_pipeline(scaler,extra)
xgb_pipe=make_pipeline(scaler,xgb)
lgb_pipe=make_pipeline(scaler,lgb)

In [14]:
# Fitting the training data 
rf_pipe.fit(X_train,y_train)
extra_pipe.fit(X_train,y_train)
xgb_pipe.fit(X_train,y_train)
lgb_pipe.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('lgbmclassifier',
                 LGBMClassifier(boosting_type='gbdt', class_weight=None,
                                colsample_bytree=1.0, importance_type='split',
                                learning_rate=0.1, max_depth=-1,
                                min_child_samples=20, min_child_weight=0.001,
                                min_split_gain=0.0, n_estimators=100, n_jobs=-1,
                                num_leaves=31, objective=None, random_state=1,
                                reg_alpha=0.0, reg_lambda=0.0, silent=True,
                                subsample=1.0, subsample_for_bin=200000,
                                subsample_freq=0))],
         verbose=False)

In [15]:
# Model Prediction
rf_pred=rf_pipe.predict(X_test)
extra_pred=extra_pipe.predict(X_test)
xgb_pred=xgb_pipe.predict(X_test)
lgb_pred=lgb_pipe.predict(X_test)

## Randomized Search for Extra_trees

In [16]:
# Defining the range of Hyperparameters for RandomSearch
n_estimators = [50,100,300,500,1000]
min_samples_split = [2,3,5,7,9]
min_samples_leaf = [1,2,4,6,8]
max_features = ['auto','sqrt','log2',None] 

In [17]:
hyperparameter_grid = {'extratreesclassifier__n_estimators': n_estimators,
                       'extratreesclassifier__min_samples_leaf': min_samples_leaf,
                       'extratreesclassifier__min_samples_split': min_samples_split,
                       'extratreesclassifier__max_features': max_features}

In [18]:
random_search=RandomizedSearchCV(extra_pipe,hyperparameter_grid,random_state=1,n_jobs=-1,verbose=1,scoring='accuracy',n_iter=10,cv=5)
random_search.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.7min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('standardscaler',
                                              StandardScaler(copy=True,
                                                             with_mean=True,
                                                             with_std=True)),
                                             ('extratreesclassifier',
                                              ExtraTreesClassifier(bootstrap=False,
                                                                   class_weight=None,
                                                                   criterion='gini',
                                                                   max_depth=None,
                                                                   max_features='auto',
                                                                   max_leaf_nodes=None,
                

In [19]:
# this code shows the best parameters of the randomsearch
random_search.best_params_

{'extratreesclassifier__n_estimators': 1000,
 'extratreesclassifier__min_samples_split': 2,
 'extratreesclassifier__min_samples_leaf': 8,
 'extratreesclassifier__max_features': None}

## Re-training Extra-tree on Best Params

In [20]:
extra2=ExtraTreesClassifier(n_estimators=1000,max_features=None,min_samples_leaf=8,min_samples_split=2)

In [21]:
extra2.fit(X_train,y_train)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=None, max_features=None, max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=8, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=1000,
                     n_jobs=None, oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

In [22]:
extra_pred2=extra2.predict(X_test)

In [23]:
metrics.accuracy_score(y_test,extra_pred2)

0.927

## |Metrics|

In [24]:
print(classification_report(y_test,rf_pred,digits=4))

precision    recall  f1-score   support

      stable     0.9191    0.8778    0.8980       712
    unstable     0.9341    0.9573    0.9456      1288

    accuracy                         0.9290      2000
   macro avg     0.9266    0.9176    0.9218      2000
weighted avg     0.9288    0.9290    0.9286      2000



In [25]:
# Accuracy Score of the light gradient boosting model
metrics.accuracy_score(y_test,lgb_pred)

0.9375

In [26]:
# Accuracy of the extreme boosting model 
metrics.accuracy_score(y_test,xgb_pred)

0.9195

## Optimized Extra_tree Feature Importance 

In [30]:
weights=extra2.feature_importances_

In [31]:
weight=pd.DataFrame(weights, index=X_train.columns)

In [34]:
print (weight.idxmax()) # Feature with the most importance 
print (weight.idxmin()) # Feature with the least importance

0    tau2
dtype: object
0    p1
dtype: object
